In [0]:
dbutils.widgets.dropdown(name = 'environment', choices= ['dev', 'uat', 'prod'], defaultValue = 'dev', label = 'select your environment')
environment = dbutils.widgets.get('environment')

In [0]:
%run "/Workspace/Users/dakuikui_outlook.com#ext#@dakuikuioutlook.onmicrosoft.com/04_common"

In [0]:
df_bronze_traffic = spark.readStream.table(f'`{environment}_catalog`.`bronze`.`raw_traffic`')

In [0]:
from pyspark.sql.functions import col, current_timestamp
def ev_count(df):
    print('Adding new column ev_count ',end = '')
    processed_df = df.withColumn('Electrics_Vehicles_Count', col('EV_Car') + col('EV_Bike'))
    print('OK')
    return processed_df

def mv_count(df):
    print('Adding new column mv_count ',end = '')
    processed_df = df.withColumn('Motor_Vehicles_Count', col('Two_wheeled_motor_vehicles') + col('Cars_and_taxis') + col('Buses_and_coaches') + col('LGV_Type') + col('HGV_Type') + col('Electrics_Vehicles_Count'))
    print('OK')
    return processed_df

def transformed_time(df):
    print('Adding new column transformed_time ',end = '')
    processed_df = df.withColumn('Transformed_Time', current_timestamp())
    print('OK')
    return processed_df


In [0]:
processed_df1 = check_duplicates(df_bronze_traffic)
processed_df2 = check_null(processed_df1, processed_df1.schema.names)
processed_df3 = ev_count(processed_df2)
processed_df4 = mv_count(processed_df3)
processed_df5 = transformed_time(processed_df4)

In [0]:
def write_silver_traffic(df,environment):
    df.writeStream\
            .format('delta')\
            .option('checkpointLocation', f'{checkpoints}/Silver_Traffic_load/checkpoint')\
            .trigger(availableNow=True)\
            .outputMode('append')\
            .toTable(f'`{environment}_catalog`.`silver`.`silver_traffic`')
    print('successfully write bronze traffic to silver table')

In [0]:
write_silver_traffic(processed_df5 ,environment)